In [3]:
import pandas as pd
import numpy as np
import ast
import re
import statistics
from sklearn.metrics import roc_auc_score

In [23]:
test_padchest = pd.read_csv('../Data/Data_splits/pathology_detection-test.csv', index_col=0)

In [4]:
preds__bbox_both_mask_inside_df = pd.read_csv("../Predictions/PD_preds_bbox_both_mask_inside.csv")

In [6]:
preds__bbox_both_mask_outside_df = pd.read_csv("../Predictions/PD_preds_bbox_both_mask_outside.csv")

In [8]:
preds__bbox_mask_inside_df = pd.read_csv("../Predictions/PD_preds_bbox_mask_inside.csv")

In [9]:
preds__bbox_mask_outside_df = pd.read_csv("../Predictions/PD_preds_bbox_mask_outside.csv")

In [10]:
preds__dilated_mask_1_inside_df = pd.read_csv("../Predictions/PD_preds_dilated_mask_1_inside.csv")

In [11]:
preds__dilated_mask_1_outside_df = pd.read_csv("../Predictions/PD_preds_dilated_mask_1_outside.csv")

In [12]:
preds__dilated_mask_2_inside_df = pd.read_csv("../Predictions/PD_preds_dilated_mask_2_inside.csv")

In [13]:
preds__dilated_mask_2_outside_df = pd.read_csv("../Predictions/PD_preds_dilated_mask_2_outside.csv")

In [14]:
preds__dilated_mask_3_inside_df = pd.read_csv("../Predictions/PD_preds_dilated_mask_3_inside.csv")

In [15]:
preds__dilated_mask_3_outside_df = pd.read_csv("../Predictions/PD_preds_dilated_mask_3_outside.csv")

In [16]:
preds__dilated_mask_4_inside_df = pd.read_csv("../Predictions/PD_preds_dilated_mask_4_inside.csv")

In [17]:
preds__dilated_mask_4_outside_df = pd.read_csv("../Predictions/PD_preds_dilated_mask_4_outside.csv")

In [19]:
preds__original_mask_inside_df = pd.read_csv("../Predictions/PD_preds_original_mask_inside.csv")

In [20]:
preds__original_mask_outside_df = pd.read_csv("../Predictions/PD_preds_original_mask_outside.csv")

In [26]:
preds__bbox_both_mask_inside_df

,Unnamed: 0,Model_name,Val_data,Preds_model1,Preds_model2,Preds_model3
0,0,"PD, Multiclass, Densenet, Imagenet, Fine-tuned",PadChest test set,[[1.30609833e-02 4.78702970e-03 4.23425436e-03...,[[4.55487594e-02 1.59274554e-03 1.64995845e-02...,[[1.04184067e-02 4.47855424e-03 5.36778290e-03...


In [44]:
# Function for reading the predictions, which are strings, as numpy arrays
def str2array(s):
    # Remove space after [
    s=re.sub('\[ +', '[', s.strip())
    # Replace commas and spaces
    s=re.sub('[,\s]+', ', ', s)
    return np.array(ast.literal_eval(s))

# Function to arrange preds nicely in a df
def get_preds_PD(orig_pred_df, true_labels_df, print_auc=False):
    
    pathologies = ['Effusion', 'Pneumothorax', 'Atelectasis', 'Cardiomegaly', 'Pneumonia']
    all_preds = []
    
    for row_number in range(len(orig_pred_df)):
        for p_idx, p in enumerate(pathologies):
            preds = [[str2array(i["Preds"]) for idx, i in orig_pred_df.iterrows()][row_number][:,p_idx]]
            all_preds.append(preds)
            
    # Constructing a df with the preds and 'true' labels

    preds_df = pd.DataFrame(list(zip(list(true_labels_df['ImagePath']),
                                     list(true_labels_df['Effusion']),
                                     list(true_labels_df['Pneumothorax']),
                                     list(true_labels_df['Atelectasis']),
                                     list(true_labels_df['Cardiomegaly']),
                                     list(true_labels_df['Pneumonia']),
                                     list(all_preds[0][0]),
                                     list(all_preds[1][0]),
                                     list(all_preds[2][0]),
                                     list(all_preds[3][0]),
                                     list(all_preds[4][0]))),
                            columns = ['ImagePath', 
                                       'Effusion', 
                                       'Pneumothorax', 
                                       'Atelectasis', 
                                       'Cardiomegaly',
                                       'Pneumonia', 
                                       'preds_Effusion_model1',
                                       'preds_Pneumothorax_model1', 
                                       'preds_Atelectasis_model1', 
                                       'preds_Cardiomegaly_model1', 
                                       'preds_Pneumonia_model1'])


    # Computing the auc for each pathology separately
    for p in pathologies:
        if print_auc:
            print(p)
        auc_list = []
        for i in range(1):
            #print(i+1)
            auc = roc_auc_score(preds_df[p], preds_df['preds_' + str(p) + '_model' + str(i+1)])
            auc_list.append(auc)
        #print(auc_list)
        
        if print_auc:
            print("Average auc:", round(sum(auc_list)/len(auc_list)*100, 1))
        
            print()

    return preds_df


In [45]:
def create_auc_table(dataframes, names):
    pathologies = ['Effusion', 'Pneumothorax', 'Atelectasis', 'Cardiomegaly', 'Pneumonia']
    columns = ["name",'Effusion', 'Pneumothorax', 'Atelectasis', 'Cardiomegaly', 'Pneumonia']
    out_dict =  {pathology:list() for pathology in columns}

    for idx in range(len(dataframes)):
        out_dict["name"].append(names[idx])

        for p in pathologies:
            auc = roc_auc_score(dataframes[idx][p], dataframes[idx]['preds_' + str(p) + '_model1'])
            out_dict[p].append(auc)

    return pd.DataFrame(out_dict)

In [46]:
create_auc_table(
    [get_preds_PD(preds__bbox_both_mask_inside_df.loc[i:i], test_padchest) for i in range(len(preds_df))],
    list(preds__bbox_both_mask_inside_df["Preds_model1	"],preds__bbox_both_mask_inside_df["Preds_model2"],preds__bbox_both_mask_inside_df["Preds_model3"])
)

ValueError: 21 columns passed, passed data had 9484 columns